### 전라남도 지역 전력소비패턴 예측

<br>

[전라남도 지역 전력소비패턴 예측](https://aifactory.space/task/2648/leaderboard)
<br>[전력신사업을 위한 전라남도 지역 전력소비패턴 데이터](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=71359)

<br>

In [1]:
import numpy as np
import sys
import pickle
import xgboost as xgb

In [2]:
x_train_path = 'data/x_train.npy'
y_train_path = 'data/y_train.npy'
model_save_path = 'output/xgboost_400.pkl'

x_train = np.load(x_train_path, allow_pickle=True)
y_train = np.load(y_train_path, allow_pickle=True)

# Convert to DMatrix
dtrain = xgb.DMatrix(x_train, label=y_train)

# Set parameters
params = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'learning_rate': 0.05,
    'subsample': 0.5,  # Consider using subsample to reduce memory usage
    'max_bin': 256,  # Reduce number of bins
    'device': 'cuda',  # Use GPU accelerated algorithm
}

# Custom evaluation function for MAPE
def mape(preds, dtrain):
    labels = dtrain.get_label()
    return 'MAPE', np.mean(np.abs((labels - preds) / (labels + 1e-6))) * 100  # Avoid division by zero

# Train the XGBoost model with custom evaluation metric
model = xgb.train(params, dtrain, evals=[(dtrain, 'train')],
                  custom_metric=mape,  # Use the custom evaluation function
                  verbose_eval=100, num_boost_round=400)

# save
pickle.dump(model, open(model_save_path, "wb"))

[0]	train-rmse:0.33601	train-MAPE:252.90921
[100]	train-rmse:0.29870	train-MAPE:171.38934
[200]	train-rmse:0.29694	train-MAPE:168.82609
[300]	train-rmse:0.29578	train-MAPE:167.22747
[399]	train-rmse:0.29498	train-MAPE:166.21057


In [3]:
import numpy as np
import sys
import pickle
import xgboost as xgb

In [5]:
model_path = 'output/xgboost_400.pkl'
x_test_path = 'data/x_test.npy'
y_pred_save_path = 'y_pred.npy'

# 모델 로딩
with open(model_path, "rb") as f:
    model = pickle.load(f)

x_test = np.load(x_test_path, allow_pickle=True)

# Convert numpy array to DMatrix
dtest = xgb.DMatrix(x_test)

y_pred = model.predict(dtest)
np.save(y_pred_save_path, y_pred)